Перейдем к задачам классификации.

In [101]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression

model_logreg = LogisticRegression()

model_logreg.fit(X_train_class_ic, y_train_class_ic)
pred_ic_class = model_logreg.predict(X_test_class_ic)

accuracy_logreg = accuracy_score(y_test_class_ic, pred_ic_class)
print(f'Accuracy: {accuracy_logreg}')

Accuracy: 0.6378737541528239


In [102]:
from sklearn.neighbors import KNeighborsClassifier

model_kneighb = KNeighborsClassifier()

model_kneighb.fit(X_train_class_ic, y_train_class_ic)
pred_ic_kneighb = model_kneighb.predict(X_test_class_ic)

accuracy_kneighb = accuracy_score(y_test_class_ic, pred_ic_kneighb)
print(f'Accuracy: {accuracy_kneighb}')

Accuracy: 0.6877076411960132


In [103]:
from sklearn.ensemble import RandomForestClassifier

model_forest = RandomForestClassifier()

model_forest.fit(X_train_class_ic, y_train_class_ic)
pred_ic_forest = model_forest.predict(X_test_class_ic)

accuracy_forest = accuracy_score(y_test_class_ic, pred_ic_forest)
print(f'Accuracy: {accuracy_forest}')

Accuracy: 0.6910299003322259


In [104]:
from xgboost import XGBClassifier

model_xg_class_ic = XGBClassifier(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

model_xg_class_ic.fit(X_train_class_ic, y_train_class_ic)
pred_xg_class_ic = model_xg_class_ic.predict(X_test_class_ic)

scores_xg_class_ic = cross_val_score(model_xg_class_ic, X_train_class_ic, y_train_class_ic, cv=5, scoring='r2')
print("Среднее R2:", scores_xg_class_ic.mean())

Среднее R2: -0.11999999999999997


Наиболее эффективно IC50 > медианы классифицирует алгоритм случайного леса (classifier). Подберем лучшие гиперпараметры для него.

In [106]:
param_dist_ic_class = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

random_search_ic_class = RandomizedSearchCV(
    estimator=RandomForestClassifier(),
    param_distributions=param_dist_ic_class,
    n_iter=50,       # сколько комбинаций попробовать
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42
)

random_search_ic_class.fit(X_train_class_ic, y_train_class_ic)

print("Лучшие параметры:", random_search_ic_class.best_params_)

Лучшие параметры: {'max_depth': 6, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 253}


In [107]:
forest_for_ic_class = RandomForestClassifier(
    n_estimators = random_search_ic_class.best_params_['n_estimators'],
    max_depth = random_search_ic_class.best_params_['max_depth'],
    min_samples_split = random_search_ic_class.best_params_['min_samples_split'],
    min_samples_leaf = random_search_ic_class.best_params_['min_samples_leaf'],
    random_state=42
)
forest_for_ic_class.fit(X_train_class_ic, y_train_class_ic)

RandomForestClassifier(max_depth=6, min_samples_leaf=5, n_estimators=253,
                       random_state=42)

In [108]:
forest_test_for_ic_class = forest_for_ic_class.predict(X_test_class_ic)  

ac_test_forest_for_ic_class = accuracy_score(y_test_class_ic, forest_test_for_ic_class)
f1_test_forest_for_ic_class = f1_score(y_test_class_ic, forest_test_for_ic_class)
prec_test_forest_for_ic_class = precision_score(y_test_class_ic, forest_test_for_ic_class)
rec_test_forest_for_ic_class = recall_score(y_test_class_ic, forest_test_for_ic_class)

print("Accuracy на тесте:", ac_test_forest_for_ic_class)
print("F1 на тесте:", f1_test_forest_for_ic_class)
print("Precision на тесте:", prec_test_forest_for_ic_class)
print("Recall на тесте:", rec_test_forest_for_ic_class)

Accuracy на тесте: 0.6843853820598007
F1 на тесте: 0.6984126984126984
Precision на тесте: 0.650887573964497
Recall на тесте: 0.7534246575342466
